## Requirements and Imports

In [ ]:
%pip install dotenv
%pip install pydantic
%pip install claudette
%pip install -U gradio
%pip install anthropic
%pip install toolslm

In [50]:
import os
from claudette import *
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import Type, get_type_hints
import inspect
import gradio as gr
import yaml
import json
import xml.etree.ElementTree as ET
import re
from anthropic.types import TextBlock, ToolUseBlock

In [47]:
import toolslm
from toolslm.funccall import *

You'll need to add your ANTHROPIC_API_KEY to a .env file to use the anthropic tools and claudette.

In [52]:
load_dotenv()

True

## Initial Settings

In [53]:
model = models[1] # Using claude-3-5-sonnet-20240620 by default

## Utils

The 3 functions in the cell below convert the json schema to yaml. I remove any unnecessary attributes to keep things clean and reduce token count.

In [54]:
def get_nested_value(obj, path):
    """
    Access nested dictionary values based on a dot-separated path.
    """
    for key in path.split('.'):
        if obj is None:
            break
        if isinstance(obj, dict):
            obj = obj.get(key)
        elif isinstance(obj, list) and key.isdigit():
            obj = obj[int(key)]
        else:
            return None
    return obj

def simplify_structure(obj, key_map):
    """
    Simplifies the structure of nested dictionaries and lists based on a mapping.
    Supports nested attributes using dot-separated paths.
    """
    if key_map is None:
        return obj
    if isinstance(obj, dict):
        return {new_key: simplify_structure(get_nested_value(obj, old_key), sub_map) 
                for old_key, new_key, sub_map in key_map if get_nested_value(obj, old_key) is not None}
    elif isinstance(obj, list):
        return [simplify_structure(item, key_map) for item in obj]
    else:
        return obj

def convert_json_to_yaml(json_obj):
    mapping = [
        ('id', 'nodeType', None),
        ('type', 'type', None),
        ('name', 'name', None),
        ('metadata.description', 'description', None),
        ('inputs', 'inputs', [
            ('name', 'name', None),
            ('type', 'type', None),
            ('metadata.defaultValue', 'defaultValue', None),
        ]),
        ('outputs', 'outputs', [
            ('name', 'name', None),
            ('type', 'type', None),
        ])
    ]

    # Apply the mapping to the provided JSON object
    simplified_json = simplify_structure(json_obj, mapping)
    # Convert the simplified structure to YAML
    yaml_data = yaml.dump(simplified_json, sort_keys=False, default_flow_style=False, allow_unicode=True)
    return yaml_data

In [55]:
# Instead of using function calls to extract structured data, claude is pretty 
# good at producing valid yaml and placing it between xml tags of our choosing.
def extract_xml_content(mixed_string, tag_name):
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    matches = re.findall(pattern, mixed_string, re.DOTALL)
    return matches

In [71]:
def create_yaml_list_from_json_nodes(nodes):
  yaml_nodes = []
  for id in nodes:
    node = [item for item in march_nodes if item['id'] == id]
    if len(node) == 0:
      continue
    node = node[0]
    yaml_nodes.append(convert_json_to_yaml(node))
  return '\n'.join([f'- {item}' for item in yaml_nodes])

In [56]:
## Import existing workflows. The files in the projects are a subset of those availalble in Magpai.
def import_workflows():
    workflows_folder = 'workflows/yaml'
    workflows = []
    for filename in os.listdir(workflows_folder):
        if filename.endswith('.yaml'):
            with open(os.path.join(workflows_folder, filename), 'r') as f:
                # workflow = yaml.safe_load(f)
                workflow = {"name": filename[:-5], "yaml": f.read() }
                workflows.append(workflow)
    return workflows

In [60]:
def get_node_ids(workflow_names):
  node_ids = []
  for wf_name in workflow_names:
    workflow_dict = [wf for wf in workflow_dicts if wf['name'] == wf_name][0]
    ids = [node['nodeType'] for node in workflow_dict['nodes']]
    node_ids.extend(ids)
  node_ids = list(set(node_ids))
  return node_ids

## Magpai Functions Setup

In [57]:
handpicked_nodes = [
  # Image
  'fal:sdxl',
  'fal:sdxl-lightning',
  # Image|Analyze
  'replicate:yolox',
  'replicate:blip_2',
  # Image|Color
  'image:colorthief',
  # Image|Create
  'default:image:noise',
  'default:image:vignette',
  'default:image:blank',
  'image:gradient:linear',
  'image:gradient:radial',
  'image:draw:shape:rectangle',
  'image:draw:shape:ellipse',
  'image:draw:shape:circle',
  # Image|Filter
  'default:image:blur',
  'default:image:drop_shadow',
  'default:image:outline',
  'default:image:filter',
  # Image|Filter|Color
  'default:image:brightness_contrast',
  # Image|Process
  'replicate:real_esrgan',
  'fal:rembg',
  # Image|Text
  'default:image:text',
  # Image|Utility
  'default:image:composite',
  'default:image:place',
  'default:image:resize',
  # Image|Utility|Mask
  'default:image:make_mask',
  'default:image:mask',
  # Outputs
  'default:output:image',
  # Text|Utility
  'default:string:format',
  # User Inputs
  'default:input:image',
]

In [58]:
# Import available nodes from Magpai. This repo includes a subset of the ~300 nodes. 
with open('data/catalogue.json') as file:
  march_nodes = json.load(file)

## Agentic Workflow

### Nodes

#### Clarify Intent 

In [59]:
workflows = import_workflows()
workflow_dicts = [yaml.safe_load(workflow['yaml']) for workflow in workflows]
workflow_descriptions = [{'name': workflow['name'], 'description': workflow['description']} for workflow in workflow_dicts]
all_nodegraph_descriptions = yaml.dump(workflow_descriptions, sort_keys=False)

In [62]:
base = "You are a node-graph building assistant. You help the user build node graphs similar to Blender, Houdini or Nuke. Given a user query, a list of potential elements, and a list of available nodes, generate a graph of nodes and edges that creates an output that satisfies the users goal."

In [63]:
clarify_goal_sp = """<reference_nodegraphs>
{nodegraph_descriptions}
</reference_nodegraphs>

{base}

Your current and only task is to help a user clarify their overall goal and the potential steps required to achieve that goal. The steps represent a node or sequence of nodes that make up a node graph. If the user asks for other information or attempts to build a graph, gently guide them back to identifying the goal and potential steps that satisfy the goal.

You can use the <reference_nodegraphs> to help inform which steps might be relevant to the users goals and the questions you ask to clarify the overall goal and steps. Do not mention the <reference_nodegraphs> in your questions.

Keep track of the overall goal and steps you have identified as well as any modifications requested by the user.

If you are not able to discern the goal or steps, ask the user to clarify! Do not attempt to guess wildly.

Ask for confirmation from the user that the overall goal and steps have been identified. Only call the relevant tool when the user confirms that they are satisfied with the overall goal and steps that have been identified."""

In [64]:
class Description(BaseModel):
    """An overall goal, key steps and summary that describes a workflow or node-graph"""
    goal: str = Field(..., description="1 sentence describing the overall goal of the workflow or node-graph.")
    keySteps: list[str] = Field(..., description="The key steps required to achieve the overall goal.")
    summary: str = Field(..., description="A summary of the steps required to achieve the overall goal.")

def extract_description(
        goal: str, # 1 sentence describing the overall goal of the workflow or node-graph
        keySteps: list[str], # The key steps required to achieve the overall goal
        summary: str # A summary of the steps required to achieve the overall goal
        ) -> dict:
    "Extract the overall goal, key steps and summary that describes a workflow or node-graph."
    return Description(goal=goal, keySteps=keySteps, summary=summary)

In [66]:
class ReferenceGraphs(BaseModel):
  names: list[str] = Field(..., description="The names of the reference graphs")

def extract_reference_graphs(
    names: list[str] # Arry of names of 3 reference graphs
    ) -> list[str]:
    "Extract the names of the reference graphs."
    return ReferenceGraphs(names=names)

In [67]:
reference_graphs_sp = """
<reference_graphs>
{all_nodegraph_descriptions}
</reference_graphs>
Based on the message history, identify the 3 most relevant node graph descriptions. Return the names of the 3 most relevant node graph descriptions."""

#### Generate Graph

In [70]:
generate_graph_prompt = """
{base}

<NodeSchemaSpecification>
  The **Node Schema** serves as a comprehensive catalog of all node types within the system, including their attributes, expected input and output types, and optional default values for inputs. This specification is foundational for defining the capabilities and interactions of different node types.

  <code language="yaml">
    nodeTypes:
    - nodeType: "NodeTypeIdentifier"
      attributes:
        name: "FunctionName"
        type: "FunctionType"
      inputs:
        - id: "InputID"
          type: "InputType"
          default: OptionalDefaultValue
      outputs:
        - id: "OutputID"
          type: "OutputType"
  </code>
  <Attributes>
  - **nodeType**: A unique identifier for the node type.
  - **attributes**: A dictionary of attributes such as `name` and `type` to describe the node's functionality.
  - **inputs** and **outputs**: Lists detailing the inputs and outputs for the node, including their types and, for inputs, optional default values.
  </Attributes>
</NodeSchemaSpecification>
<GraphRepresentationSpecification>
  The **Graph Representation** focuses on illustrating the interconnectedness of nodes within a graph. Each node is referenced by a unique identifier and its type, as defined in the Node Schema. This abstraction allows for a high-level view of the graph's structure and data flow.

  <code language="yaml">
    nodes:
      - id: "UniqueNodeIdentifier"
        nodeType: "NodeTypeIdentifier"
        inputs:
          - name: "InputName"
            value: InputValue

    edges:
      - source: "SourceNodeIdentifier.OutputID"
        target: "TargetNodeIdentifier.InputID"
  </code>

  <Components>
  - **nodes**: A list where each entry specifies a node in the graph, identified by a unique `id` and a `nodeType` that corresponds to an entry in the Node Schema.
  - **edges**: Defined by linking the outputs of one or more nodes to the inputs of one or more other nodes.
  </Components>
</GraphRepresentationSpecification>
<NodeSchema>
  <code language="yaml">
    nodeTypes:
    {yaml_nodes}
  </code>
</NodeSchema>

<reference_nodegraphs>
{reference_nodegraphs}
</reference_nodegraphs>

<Goal>{goal_description}</Goal>

<Instructions>
Identify the elements from the <Goal>. Start with each element and progressively work forward in generating the graph network. For each new node added to the graph:
1. Start each step with an H2 header with the step number - eg. `## Step 1`
2. For each step, use an H3 header, `### Thought`, to capture information about which node will be used and the purpose fo that node in building the graph.
3. After the `### Thought` header use an H3 header, `### Act`, to capture the update to the graph. This contains the node id and nodeType in yaml format. If relevant, include the edge source and target in yaml format.
4. After the `### Act` header use an H3 header, `### Observe`, to describe the current state of the graph.
5. Once the graph network has been completed, wrap the final output in `<FinalGraph>` tags.
</Instructions>

<Guidelines>
- You can use the <reference_nodegraphs> to help inform what to do at each step.
- Favour image generation nodes over input nodes.
- Always provide a prompt for the image generation nodes.
- Ensure that the text nodes have the correct text content.
- Ensure that midground and foreground elements have transparent backgrounds by using a remove background node after an image generation node.
- A target attribute can only be connected to a maximum of one source attribute.
</Guidelines>

Here is an example of the expected output:

<example>
Certainly! I'll create a graph based on the request, including the elements that have been identified. Let's start building the graph step by step.
## Step 1
### Thought
The fal:sdxl node will be used to generate a background image. The prompt will be, 'An image of a green field that is out of focus. Heavily blurred'.
### Act
```yaml
nodes:
  - id: "Background"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "An image of a green field that is out of focus. Heavily blurred"
```
### Observe
The current graph includes a single node that generates a background image.

## Step 2
### Thought
The default:image:composite node will be used to layer an image of poppies over the background image. The background image will be connected to the under input attribute.
### Act
```yaml
nodes:
  - id: "Background"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "An image of a green field that is out of focus. Heavily blurred"
  - id: "Composite1"
    nodeType: "default:image:composite"
edges:
  - source: "Background.image"
    target: "Composite1.under"
```
### Observe
The graph currently has a background image connected to a composite node. The composite node requires another image to be connected to the over input attribute.

## Step 3
### Thought
The fal:sdxl node will be used to generate an image of poppies. The prompt will be, 'A bunch of poppies'.
### Act
```yaml
nodes:
  - id: "Background"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "An image of a green field that is out of focus. Heavily blurred"
  - id: "Composite1"
    nodeType: "default:image:composite"
  - id: "Poppies"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "A bunch of poppies"
edges:
  - source: "Background.image"
    target: "Composite1.under"
  - source: "Poppies.image"
    target: "Composite1.over"
```
### Observe
The current graph composites an image of poppies over a blurred background.

## Step 4
### Thought
The default:output:image node will be used to export the final result of the graph. This node will always be at the end of the node tree.
### Act
```yaml
nodes:
  - id: "Background"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "An image of a green field that is out of focus. Heavily blurred"
  - id: "Composite1"
    nodeType: "default:image:composite"
  - id: "Poppies"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "A bunch of poppies"
  - id: "Output1"
    nodeType: "default:output:image"
edges:
  - source: "Background.image"
      target: "Composite1.under"
  - source: "Poppies.image"
      target: "Composite1.over"
  - source: "Composite1.image"
      target: "Output1.image"
```
### Observe
The graph exports an image of a bunch of poppies composited over a blurred background.

## Final Graph
Here is the final graph:
<FinalGraph>
```yaml
nodes:
  - id: "Background"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "An image of a green field that is out of focus. Heavily blurred"
  - id: "Composite1"
    nodeType: "default:image:composite"
  - id: "Poppies"
    nodeType: "fal:sdxl"
    inputs:
      - name: "prompt"
        value: "A bunch of poppies"
  - id: "Output1"
    nodeType: "default:output:image"
edges:
  - source: "Background.image"
    target: "Composite1.under"
  - source: "Poppies.image"
    target: "Composite1.over"
  - source: "Composite1.image"
    target: "Output1.image"
```
</FinalGraph>
</example>
"""

#### Validate Graph

In [72]:
validate_graph_prompt = """
{base}

<NodeSchemaSpecification>
  The **Node Schema** serves as a comprehensive catalog of all node types within the system, including their attributes, expected input and output types, and optional default values for inputs. This specification is foundational for defining the capabilities and interactions of different node types.

  <code language="yaml">
    nodeTypes:
    - nodeType: "NodeTypeIdentifier"
      attributes:
        name: "FunctionName"
        type: "FunctionType"
      inputs:
        - id: "InputID"
          type: "InputType"
          default: OptionalDefaultValue
      outputs:
        - id: "OutputID"
          type: "OutputType"
  </code>
  <Attributes>
  - **nodeType**: A unique identifier for the node type.
  - **attributes**: A dictionary of attributes such as `name` and `type` to describe the node's functionality.
  - **inputs** and **outputs**: Lists detailing the inputs and outputs for the node, including their types and, for inputs, optional default values.
  </Attributes>
</NodeSchemaSpecification>
<GraphRepresentationSpecification>
  The **Graph Representation** focuses on illustrating the interconnectedness of nodes within a graph. Each node is referenced by a unique identifier and its type, as defined in the Node Schema. This abstraction allows for a high-level view of the graph's structure and data flow.

  <code language="yaml">
    nodes:
      - id: "UniqueNodeIdentifier"
        nodeType: "NodeTypeIdentifier"
        inputs:
          - name: "InputName"
            value: InputValue

    edges:
      - source: "SourceNodeIdentifier.OutputID"
        target: "TargetNodeIdentifier.InputID"
  </code>

  <Components>
  - **nodes**: A list where each entry specifies a node in the graph, identified by a unique `id` and a `nodeType` that corresponds to an entry in the Node Schema.
  - **edges**: Defined by linking the outputs of one or more nodes to the inputs of one or more other nodes, or by specifying values directly to inputs.
  </Components>
</GraphRepresentationSpecification>
<NodeSchema>
  <code language="yaml">
    nodeTypes:
    {yaml_nodes}
  </code>
</NodeSchema>

<Goal>{goal_description}</Goal>

<GraphToValidate>
{graph}
</GraphToValidate>

<Instructions>
Validate the generated graph by ensuring that all elements are included and correctly connected. Start from the final output node and work backwards to ensure that all elements are present and correctly linked. If any issues are found, make the necessary corrections and re-validate the graph.

1. Wrap your analysis in opening and closing `<thinking>` tags.
2. After you have identified any issues, generate an updated graph network wrapped in `<FinalGraph>` tags.
</Instructions>

<Guidelines>
- Favour image generation nodes over input nodes.
- Always provide a prompt for the image generation nodes.
- Ensure that the text nodes have the correct text content.
- Ensure that midground and foreground elements have transparent backgrounds by using a remove background node after an image generation node.
- A target attribute can only be connected to a maximum of one source attribute.
</Guidelines>

Here is an example of the expected output:

<example>
Let's make sure the graph is valid. I'll check the graph based on the goal and schema specification. Let's validate the graph step by step.
<thinking>
It appears the graph has some issues:
- issue1
- issue2
</thinking>
Here's an updated version of the graph:
<FinalGraph>
```yaml
nodes:
- id: "Background"
  nodeType: "fal:sdxl"
  inputs:
    - name: "prompt"
      value: "An image of a green field that is out of focus. Heavily blurred"
- id: "Composite1"
  nodeType: "default:image:composite"
- id: "Poppies"
  nodeType: "fal:sdxl"
  inputs:
    - name: "prompt"
      value: "A bunch of poppies"
- id: "Output1"
  nodeType: "default:output:image"
edges:
  - source: "Background.image"
    target: "Composite1.under"
  - source: "Poppies.image"
    target: "Composite1.over"
  - source: "Composite1.image"
    target: "Output1.image"
```
</FinalGraph>
</example>
"""

## Graph Agent Class

In [87]:
class State(BaseModel):
  elements: list[str]
  user_query: str
  yaml_nodes: str
  graph: str
  ref_graphs: list[str]
  description: Description

In [88]:
class GraphAgent():
  def __init__(self,
               clarify_goal_sp: str,
               reference_graphs_sp: str,
               validate_graph_sp: str,
               generate_graph_sp: str):
    self.clarify_goal_sp = clarify_goal_sp
    self.reference_graphs_sp = reference_graphs_sp
    self.validate_graph_sp = validate_graph_sp
    self.generate_graph_sp = generate_graph_sp
    self.fast_model = models[2]
    self.model = model

    self.goal_chat = Chat(self.model, 
                          sp=self.clarify_goal_sp.format(base=base, nodegraph_descriptions=""), tools=[extract_description])

    self.ref_chat = Chat(self.fast_model,
                         sp=self.reference_graphs_sp.format(all_nodegraph_descriptions=all_nodegraph_descriptions),
                         tools=[extract_reference_graphs], 
                         tool_choice="extract_reference_graphs")

    self.graph_gen_llm = Chat(model)

    self.validate_graph_llm = Chat(model)

    self.state = State(elements=[], 
                       user_query="", 
                       yaml_nodes="", 
                       graph="",
                       ref_graphs=[],
                       description=Description(
                         goal="",
                         keySteps=[],
                         summary=""
                       ))

  def _generate_yaml_list(self):
    ref_ids = get_node_ids(self.state.ref_graphs)
    combined_ids = list(set(handpicked_nodes + ref_ids)) 
    yaml_list = create_yaml_list_from_json_nodes(combined_ids)

    self.state.yaml_nodes = yaml_list
  
  def _generate_graph(self):
    self.graph_gen_llm.h = [] # Clear the chat history

    # Create a string of the identified reference graphs 
    reference_nodegraphs = yaml.dump([wf for wf in workflow_dicts if wf['name'] in self.state.ref_graphs], sort_keys=False)

    goal_description = yaml.dump(self.state.description.model_dump(), sort_keys=False)

    self._generate_yaml_list()

    r = self.graph_gen_llm(
      self.generate_graph_sp.format(
        base=base,
        yaml_nodes=self.state.yaml_nodes,
        reference_nodegraphs=reference_nodegraphs,
        goal_description=goal_description),
        stream=True)
    yield from r

    content = [blk.text for blk in self.graph_gen_llm.c.result.content if isinstance(blk, TextBlock)][0]
    if content:
      xml_content = extract_xml_content(content, 'FinalGraph')
      if len(xml_content) > 0:
        final_graph_content = xml_content[0].strip()
        # Remove ```yaml from beginning and ``` from end
        final_graph_content = final_graph_content[8:-3]
      else:
        # TODO: Should probably return an error
        final_graph_content = ""
      self.state.graph = final_graph_content

  def _validate_graph(self):
    self.validate_graph_llm.h = [] # Clear the chat history

    goal_description = yaml.dump(self.state.description.model_dump(), sort_keys=False)

    self._generate_yaml_list()

    r = self.validate_graph_llm(
      self.validate_graph_sp.format(
        base=base,
        yaml_nodes=self.state.yaml_nodes,
        goal_description=goal_description,
        graph=self.state.graph),
        stream=True)
    yield from r

    content = [blk.text for blk in self.validate_graph_llm.c.result.content if isinstance(blk, TextBlock)][0]
    if content:
      xml_content = extract_xml_content(content, 'FinalGraph')
      if len(xml_content) > 0:
        final_graph_content = xml_content[0].strip()
        # Remove ```yaml from beginning and ``` from end
        final_graph_content = final_graph_content[8:-3]
      else:
        final_graph_content = ""
      self.state.graph = final_graph_content


  def __call__(self, message ):
    # Set the ref chat history to the goal chat history if there's an existing coversation
    if len(self.goal_chat.h) > 0:
      history = self.goal_chat.h
      if history[-1]['role'] == 'assistant':
        history = history[:-1]
      self.ref_chat.h = history

      # Identify reference graphs based on the current chat history
      self.ref_chat()
      for ref_blk in self.ref_chat.c.result.content:
        if isinstance(ref_blk, ToolUseBlock):
          self.state.ref_graphs = ref_blk.input['names']

    # Create a string of the identified reference graph descriptions 
    nodegraph_descriptions = yaml.dump([wf for wf in workflow_descriptions if wf['name'] in self.state.ref_graphs], sort_keys=False)

    # Update the goal system prompt to include the ref graph descriptions
    self.goal_chat.sp = clarify_goal_sp.format(base=base, nodegraph_descriptions=nodegraph_descriptions)

    goal_response = self.goal_chat(message, stream=True)
    partial_message = ""
    for o in goal_response:
      partial_message += o
      yield partial_message

    for goal_blk in self.goal_chat.c.result.content:
      if isinstance(goal_blk, ToolUseBlock):
        self.state.description = Description(**goal_blk.input)
        partial_message += f"\n\nWe're ready to proceed with the following description:\nGoal:\n{self.state.description.goal}\nKey Steps:\n- " + "\n- ".join(self.state.description.keySteps) + f"\nSummary:\n{self.state.description.summary}\n"
        yield partial_message

        graph_response = self._generate_graph()
        for o in graph_response:
          partial_message += o
          yield partial_message
        partial_message += "\n\n"
        yield partial_message 

        validate_response = self._validate_graph()
        for o in validate_response:
          partial_message += o
          yield partial_message
        partial_message += "\n\n"
        yield partial_message 
        
        partial_message += f"Here's the final graph:\n```yaml\n{self.state.graph}\n```"
        yield partial_message

In [89]:
graph_agent = GraphAgent(
  clarify_goal_sp=clarify_goal_sp,
  reference_graphs_sp=reference_graphs_sp,
  validate_graph_sp=validate_graph_prompt,
  generate_graph_sp=generate_graph_prompt
)

In [ ]:
demo_theme = gr.themes.Soft(
    primary_hue="slate",
    secondary_hue="sky",
    text_size="lg",
    spacing_size="lg",
    font=[gr.themes.GoogleFont('Solway'), 'ui-sans-serif', 'system-ui', 'sans-serif'],
).set(
    body_background_fill='*primary_200',
    shadow_drop='*shadow_spread'
)

with gr.Blocks(theme=demo_theme) as demo:
  with gr.Row():
    with gr.Column(scale=2):
      chatbot = gr.Chatbot(show_label=False,
                           label="Chatbot",
                           height="63vh")
      with gr.Group():
        with gr.Row():
          msg = gr.Textbox(container=False,
                           show_label=False,
                           label="chat_box",
                           placeholder="Type a message...",
                           scale=7)
          submit = gr.Button("Submit",
                             variant="primary",
                             scale=1)
      clear = gr.Button("Clear")
    with gr.Column(variant="panel", scale=1):
      textbox = gr.Textbox(label="Info", interactive=False)
  
    # If you want to print out debug info after each run, add it to the formatted_summary. Currently the reference graphs are being printed out
    def update_infobox(history):
      if len(graph_agent.state.ref_graphs) > 0:
        ref_summaries = [(ref_graph['name'], ref_graph['description']['summary']) for ref_graph in workflow_dicts if ref_graph['name'] in graph_agent.state.ref_graphs]
        formatted_summary = "## Reference graphs:\n" + "\n".join([f"{name}: {summary}\n" for name, summary in ref_summaries])
        return formatted_summary

      return ""

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def msg_handler(history):
      message = history[-1][0]
      response = graph_agent(message)
      history[-1][1] = ""
      for o in response:
        history[-1][1] = o
        yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        msg_handler, chatbot, chatbot
    ).then(update_infobox, chatbot, textbox)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

In [91]:
demo.close()

Closing server running on port: 7869


## Workflow Analysis

In [22]:
# Import workflows
def import_workflows():
    workflows_folder = 'workflows/yaml'
    workflows = []
    for filename in os.listdir(workflows_folder):
        if filename.endswith('.yaml'):
            with open(os.path.join(workflows_folder, filename), 'r') as f:
                # workflow = yaml.safe_load(f)
                workflow = {"name": filename[:-5], "yaml": f.read() }
                workflows.append(workflow)
    return workflows

In [23]:
workflows = import_workflows()

In [24]:
analyse_prompt = """
You are a helpful node graph analyser. Given a node graph and a list of node schemas you describe the overall goal of the node graph and how that goal is achieved.

<NodeSchemaSpecification>
  The **Node Schema** serves as a comprehensive catalog of all node types within the system, including their attributes, expected input and output types, and optional default values for inputs. This specification is foundational for defining the capabilities and interactions of different node types.

  <code language="yaml">
    nodeTypes:
    - nodeType: "NodeTypeIdentifier"
      attributes:
        name: "FunctionName"
        type: "FunctionType"
      inputs:
        - id: "InputID"
          type: "InputType"
          default: OptionalDefaultValue
      outputs:
        - id: "OutputID"
          type: "OutputType"
  </code>
  <Attributes>
  - **nodeType**: A unique identifier for the node type.
  - **attributes**: A dictionary of attributes such as `name` and `type` to describe the node's functionality.
  - **inputs** and **outputs**: Lists detailing the inputs and outputs for the node, including their types and, for inputs, optional default values.
  </Attributes>
</NodeSchemaSpecification>
<GraphRepresentationSpecification>
  The **Graph Representation** focuses on illustrating the interconnectedness of nodes within a graph. Each node is referenced by a unique identifier and its type, as defined in the Node Schema. This abstraction allows for a high-level view of the graph's structure and data flow.

  <code language="yaml">
    nodes:
      - id: "UniqueNodeIdentifier"
        nodeType: "NodeTypeIdentifier"
      inputs:
        - name: "InputName"
          value: InputValue

    edges:
      - source: "SourceNodeIdentifier.OutputID"
        target: "TargetNodeIdentifier.InputID"
  </code>

  <Components>
  - **nodes**: A list where each entry specifies a node in the graph, identified by a unique `id` and a `nodeType` that corresponds to an entry in the Node Schema.
  - **edges**: Defined by linking the outputs of one or more nodes to the inputs of one or more other nodes, or by specifying values directly to inputs.
  </Components>
</GraphRepresentationSpecification>
<NodeSchema>
  <code language="yaml">
    nodeTypes:
    {yaml_nodes}
  </code>
</NodeSchema>
<analysis_instructions>
1. The structure of the response should be yaml as follows:
  - 1 sentence describing the overall goal
  - 3-8 bullet points describing the key steps
  - 1-2 sentence summary
2. Wrap the analysis in opening and closing `<analysis>` tags.
</analysis_instructions>
<example>
  <assistant_response>
    <analysis>
    description:
      - goal: Create a visually striking image featuring a random news headline with a complementary background.
      - keySteps:
        - Fetch and select a random news headline and image
        - Process the image and extract its dominant color
        - Shorten the headline using AI
        - Generate a gradient overlay based on the image's color
        - Compose the final image by layering the processed elements
        - Output the result as a JPG image
      - summary: This workflow combines data fetching, image processing, and AI-driven text manipulation to transform a news article into an eye-catching visual representation. It effectively balances automation with design principles to create a shareable news graphic.
    </analysis>
  </assistant_response>
</example>
"""

In [27]:
def analyse_workflows(workflows):
    for workflow in workflows:
        workflow_dict = yaml.safe_load(workflow['yaml'])
        workflow_nodes = [node['nodeType'] for node in workflow_dict['nodes']]
        workflow_yaml_list = create_yaml_list_from_json_nodes(workflow_nodes)
        chat = Chat(model, sp=analyse_prompt.format(yaml_nodes=workflow_yaml_list))
        r = chat(f"Please analyse the following workflow:\n## {workflow['name']}\n {workflow['yaml']}")
        analysis = extract_xml_content(r.content[0].text, 'analysis')[0]

        # Open the yaml file and prepend the name and analyis to the yaml
        with open(f'workflows/yaml/{workflow["name"]}.yaml', 'w') as f:
            f.write('name: ' + workflow['name'] + '\n' + analysis + '\n' + workflow['yaml'])

In [28]:
analyse_workflows(workflows)

In [87]:
chat = Chat(model, sp=analyse_prompt.format(yaml_nodes=workflow_yaml_list))
r = chat(f"Please analyse the following workflow:\n## {workflow['name']}\n {workflow['yaml']}")
r

<analysis>
description:
  - goal: Generate a visually appealing YouTube thumbnail featuring flying pigs based on a user-provided video description.

  - keySteps:
    - Accept user input for the video description
    - Generate multiple image ideas using GPT-4 based on the description
    - Select a specific image idea from the generated list
    - Create a high-quality image using DALL-E 3 based on the selected idea
    - Process the generated image by resizing, adding an outline, and applying a drop shadow
    - Compose the final thumbnail by combining the processed image with additional elements

  - summary: This workflow automates the creation of an eye-catching YouTube thumbnail by leveraging AI-generated content and image processing techniques. It combines user input, text generation, image creation, and various image manipulation steps to produce a professional-looking thumbnail tailored to the video's content.
</analysis>

<details>

- id: msg_01CQU7qbCbZLr2GiCGYh5Ltw
- content: [{'text': "<analysis>\ndescription:\n  - goal: Generate a visually appealing YouTube thumbnail featuring flying pigs based on a user-provided video description.\n\n  - keySteps:\n    - Accept user input for the video description\n    - Generate multiple image ideas using GPT-4 based on the description\n    - Select a specific image idea from the generated list\n    - Create a high-quality image using DALL-E 3 based on the selected idea\n    - Process the generated image by resizing, adding an outline, and applying a drop shadow\n    - Compose the final thumbnail by combining the processed image with additional elements\n\n  - summary: This workflow automates the creation of an eye-catching YouTube thumbnail by leveraging AI-generated content and image processing techniques. It combines user input, text generation, image creation, and various image manipulation steps to produce a professional-looking thumbnail tailored to the video's content.\n</analysis>", 'type': 'text'}]
- model: claude-3-5-sonnet-20240620
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 3270, 'output_tokens': 203}

</details>